In [3]:
import torch
from transformers import BertTokenizer
from BertRegressor import BertRegressor

In [4]:
# Define the model architecture and load the trained weights

model = BertRegressor()
model.load_state_dict(torch.load('bert_model.pt'))
model.eval()

BertRegressor(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_aff

In [5]:
# Load the tokenizer and tokenize the input text

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

playlist_name = "chill rab mix"
encoded_playlist_name = tokenizer(text = playlist_name, 
                           add_special_tokens=True,
                           padding = 'max_length',
                           truncation = 'longest_first',
                           max_length = 300,
                           return_attention_mask = True)

input_ids = encoded_playlist_name['input_ids']
attention_masks = encoded_playlist_name['attention_mask']

In [9]:
# run the model on the input text
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
print(f"using device: {device}")

with torch.no_grad():
    output = model(torch.tensor(input_ids).to(device).unsqueeze(0), 
                   torch.tensor(attention_masks).to(device).unsqueeze(0))
    
    print(f"predicted playlist score: {output.tolist()}")

using device: cuda
predicted playlist score: [[0.2946215569972992, 0.515164852142334, -0.07495331764221191, -0.3546474575996399, -0.15018747746944427, 0.04391321539878845, -0.406247079372406, 0.1349208652973175, -1.0565917491912842, 0.21842052042484283, -0.7120060324668884, 0.28946053981781006, -0.3755081593990326]]


In [33]:
# define functions that have more generatlized use

import torch
from torch.utils.data import TensorDataset, DataLoader

def predict(model, dataloader, device):
    model.eval()
    output = []
    for batch in dataloader:
        batch_inputs, batch_masks = \
                                  tuple(b.to(device) for b in batch)
        with torch.no_grad():
            output += model(batch_inputs, 
                            batch_masks).tolist()
    return output


def create_dataloaders(inputs, masks, batch_size=13):
    input_tensor = torch.tensor(inputs)
    mask_tensor = torch.tensor(masks)
    dataset = TensorDataset(input_tensor, mask_tensor)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    return dataloader


def prepare_inputs(playlist_names, tokenizer):
    encoded_playlist_names = tokenizer(text = playlist_names, 
                                       add_special_tokens=True,
                                       padding = 'max_length',
                                       truncation = 'longest_first',
                                       max_length = 300,
                                       return_attention_mask = True)
    
    input_ids = encoded_playlist_names['input_ids']
    attention_masks = encoded_playlist_names['attention_mask']
    
    return input_ids, attention_masks

In [34]:
songs = ["chill rap mix", "coffee shop", "aggressive rap and drill"]
input_ids, attention_masks = prepare_inputs(songs, tokenizer)
dataloader = create_dataloaders(input_ids, attention_masks)
prediction = predict(model, dataloader, device)

print("predicted playlist scores: ")
for idx, song in enumerate(songs):
    print(f"{song}: {prediction[idx]}")


predicted playlist scores: 
chill rap mix: [1.583203673362732, 0.07741276174783707, -0.6467263698577881, -1.691370964050293, 1.3265907764434814, 0.06018896400928497, -1.2053993940353394, -1.211018681526184, -0.01895637810230255, -0.5510643720626831, -1.0612151622772217, -0.8414212465286255, -0.3889871835708618]
coffee shop: [-0.7125457525253296, 1.0681607723236084, -0.2963963747024536, 0.22406548261642456, -0.47823041677474976, -0.41923460364341736, 0.4840065538883209, 0.42777150869369507, -0.23099486529827118, 1.988478422164917, 0.9496800899505615, 0.7885478138923645, -0.3059929609298706]
aggressive rap and drill: [-0.29510995745658875, 0.941478967666626, -0.2874526381492615, -0.05669371038675308, -0.4419631361961365, 0.2741410434246063, 0.1859968602657318, 0.23342745006084442, -0.9475931525230408, 1.5186402797698975, -0.07284444570541382, 0.48750314116477966, -0.27869996428489685]


In [36]:
songs = ["coffee shop"]
input_ids, attention_masks = prepare_inputs(songs, tokenizer)
dataloader = create_dataloaders(input_ids, attention_masks)

print("predicted playlist scores: ")
for idx, song in enumerate(songs):
    print(f"{song}: {prediction[idx]}")

predicted playlist scores: 
coffee shop: [1.583203673362732, 0.07741276174783707, -0.6467263698577881, -1.691370964050293, 1.3265907764434814, 0.06018896400928497, -1.2053993940353394, -1.211018681526184, -0.01895637810230255, -0.5510643720626831, -1.0612151622772217, -0.8414212465286255, -0.3889871835708618]
